In [3]:
import pandas as pd
import re
import string
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np

WELLS = 384

In [4]:
nav = pd.read_csv('./input/NSC_IV_ppm120_18.13.14 RF091120.csv',sep='\t', index_col = 0)

In [5]:
nav.head()

,Unnamed: 1,Sweep 001,Sweep 001.1,Sweep 001.2,Sweep 001.3,Sweep 001.4,Sweep 001.5,Sweep 001.6,Sweep 001.7,Sweep 002,...,Sweep 010.6,Sweep 010.7,Sweep 011,Sweep 011.1,Sweep 011.2,Sweep 011.3,Sweep 011.4,Sweep 011.5,Sweep 011.6,Sweep 011.7
Sweep Results,,,,,,,,,,,,,,,,,,,,,
Parameter,Valid QC,Compound Name,Seal Resistance,INa,IK,Itail,Capacitance,Sweep Voltage/Current,Sweep disregarded,Compound Name,...,Sweep Voltage/Current,Sweep disregarded,Compound Name,Seal Resistance,INa,IK,Itail,Capacitance,Sweep Voltage/Current,Sweep disregarded
Time of Sweep,NaN,4.2110,4.2110,4.2110,4.2110,4.2110,4.2110,4.2110,4.2110,14.2700,...,94.6780,94.6780,104.7190,104.7190,104.7190,104.7190,104.7190,104.7190,104.7190,104.7190
A01,T,External,2.342842E+9,-9.101119E-12,1.971909E-11,-8.646063E-11,1.726758E-11,-8.000000E-2,F,SE,...,5.500000E-2,F,FullBlock,2.097781E+9,-1.122471E-10,8.312355E-10,4.080335E-10,1.731280E-11,7.000000E-2,F
B01,F,External,6.206835E+6,1.805055E-10,2.078089E-10,4.838762E-10,NaN,-8.000000E-2,F,SE,...,5.500000E-2,F,FullBlock,6.273440E+6,5.612357E-10,6.203930E-10,4.049998E-10,NaN,7.000000E-2,F
C01,T,External,3.274427E+9,-1.820224E-11,1.365168E-11,-1.198314E-10,1.255932E-11,-8.000000E-2,F,SE,...,5.500000E-2,F,FullBlock,2.587171E+9,7.887636E-11,4.717413E-10,2.078089E-10,1.283219E-11,7.000000E-2,F


In [10]:
nav.iloc['A01',1] # REMEMBER TO FILL NaN

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [11]:
index = nav.index
columns = nav.columns

In [12]:
num_sweeps = 1
num_param = 0
bool_mask = []

temp = 0
for column in columns:
    if re.match("Sweep \d{3}\.\d", column):
        bool_mask.append(True)
        param = int(column.split(".")[-1])
        if param > temp:
            temp = param
            if num_sweeps == 1:
                num_param += 1
        else:
            temp = 0
            num_sweeps += 1
    else:
        bool_mask.append(False)
        

In [13]:
num_param,num_sweeps

(7, 11)

In [14]:
index_first_param = 2
columns[index_first_param]

'Sweep 001.1'

In [15]:
rel_data = nav.loc['A01':, columns[bool_mask]]

In [16]:
plate_cols = 24
plate_rows = 16
row_names = list(string.ascii_uppercase[:plate_rows])

In [17]:
rel_data.head(20)

,Sweep 001.1,Sweep 001.2,Sweep 001.3,Sweep 001.4,Sweep 001.5,Sweep 001.6,Sweep 001.7,Sweep 002.1,Sweep 002.2,Sweep 002.3,...,Sweep 010.5,Sweep 010.6,Sweep 010.7,Sweep 011.1,Sweep 011.2,Sweep 011.3,Sweep 011.4,Sweep 011.5,Sweep 011.6,Sweep 011.7
Sweep Results,,,,,,,,,,,,,,,,,,,,,
A01,2.342842E+9,-9.101119E-12,1.971909E-11,-8.646063E-11,1.726758E-11,-8.000000E-2,F,2.302630E+9,-1.516853E-11,1.668538E-11,...,1.749345E-11,5.500000E-2,F,2.097781E+9,-1.122471E-10,8.312355E-10,4.080335E-10,1.731280E-11,7.000000E-2,F
B01,6.206835E+6,1.805055E-10,2.078089E-10,4.838762E-10,NaN,-8.000000E-2,F,6.204210E+6,1.805055E-10,2.153931E-10,...,NaN,5.500000E-2,F,6.273440E+6,5.612357E-10,6.203930E-10,4.049998E-10,NaN,7.000000E-2,F
C01,3.274427E+9,-1.820224E-11,1.365168E-11,-1.198314E-10,1.255932E-11,-8.000000E-2,F,2.836806E+9,-1.820224E-11,1.668538E-11,...,1.331661E-11,5.500000E-2,F,2.587171E+9,7.887636E-11,4.717413E-10,2.078089E-10,1.283219E-11,7.000000E-2,F
D01,5.429501E+9,-1.820224E-11,1.516853E-11,1.274157E-10,1.717139E-11,-8.000000E-2,F,6.385435E+9,-3.033706E-11,1.668538E-11,...,1.796226E-11,5.500000E-2,F,6.258189E+9,5.536514E-10,1.201348E-9,3.822470E-10,1.718783E-11,7.000000E-2,F
E01,1.412725E+8,7.584266E-12,4.247189E-11,2.578650E-11,NaN,-8.000000E-2,F,1.471854E+8,-1.516853E-12,3.943818E-11,...,2.328658E-13,5.500000E-2,F,1.821499E+8,-8.949434E-11,5.005615E-11,2.578650E-11,NaN,7.000000E-2,F
F01,2.426085E+9,-1.516853E-11,1.365168E-11,-6.674154E-11,1.350609E-11,-8.000000E-2,F,6.940433E+9,-1.061797E-11,2.426965E-11,...,1.389927E-11,5.500000E-2,F,2.378214E+9,1.031460E-10,6.719659E-10,2.608987E-10,1.360137E-11,7.000000E-2,F
G01,2.508729E+9,-1.365168E-11,2.426965E-11,-1.880898E-10,3.832494E-11,-8.000000E-2,F,2.521548E+9,-1.820224E-11,1.820224E-11,...,3.886387E-11,5.500000E-2,F,2.575118E+9,-6.825839E-11,4.049998E-10,1.729213E-10,3.864949E-11,7.000000E-2,F
H01,4.202579E+9,-1.213483E-11,1.820224E-11,3.185392E-11,3.073383E-12,-8.000000E-2,F,3.825604E+9,-1.516853E-11,1.971909E-11,...,3.588518E-12,5.500000E-2,F,4.276720E+9,1.516853E-12,1.471348E-10,7.280895E-11,3.368715E-12,7.000000E-2,F
I01,3.060483E+9,-1.213483E-11,1.820224E-11,-2.730336E-11,6.124837E-12,-8.000000E-2,F,2.853207E+9,-1.668538E-11,2.275280E-11,...,6.030568E-12,5.500000E-2,F,2.901130E+9,4.550560E-11,5.111795E-10,1.956741E-10,5.831910E-12,7.000000E-2,F


In [18]:
clean = pd.DataFrame(data = rel_data.iloc[:,0].to_numpy().reshape(plate_rows, plate_cols,order='F'), index=row_names, columns=range(1,plate_cols+1))

In [19]:
clean.head()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,2.342842E+9,1.963912E+9,2.738740E+9,1.183055E+10,3.492943E+9,1.120052E+9,6.183113E+6,1.581740E+9,2.078220E+9,1.309771E+9,...,1.340375E+7,1.654662E+9,3.732599E+9,1.008325E+9,5.357176E+7,4.683894E+7,1.128191E+9,4.430292E+9,1.205766E+9,2.372203E+9
B,6.206835E+6,2.856156E+9,1.938836E+9,2.886501E+9,2.410271E+9,1.615961E+9,3.240638E+9,6.044915E+8,1.253682E+9,1.550296E+8,...,6.367779E+9,5.629781E+9,4.460323E+9,6.154405E+6,6.189346E+6,1.256075E+9,1.438335E+9,4.345056E+9,1.480705E+8,6.618806E+8
C,3.274427E+9,1.097544E+10,3.961061E+8,4.233481E+9,6.354624E+6,1.813635E+9,6.525350E+8,1.760265E+9,7.298026E+8,3.937897E+9,...,1.367179E+9,4.570983E+9,1.249083E+8,1.338832E+9,1.400576E+9,7.104412E+9,4.961609E+9,2.171630E+9,9.414706E+6,6.940272E+9
D,5.429501E+9,1.754009E+9,5.625113E+7,8.471114E+8,9.828885E+8,2.310330E+9,8.362459E+7,3.087839E+9,5.795446E+8,1.662026E+9,...,1.376714E+9,2.268120E+9,9.347910E+8,9.627313E+8,3.837290E+9,4.247310E+7,9.170481E+8,2.079736E+7,4.581970E+7,2.660905E+9
E,1.412725E+8,7.931853E+8,9.716541E+7,1.265276E+9,7.990178E+7,5.412488E+9,3.346593E+9,6.013090E+9,3.559526E+9,6.197703E+6,...,6.527200E+9,4.378353E+9,3.652678E+9,5.617563E+8,6.364847E+9,5.563955E+8,2.696737E+9,7.650784E+8,1.571665E+9,2.116092E+9


In [12]:
clean.to_csv('clean.csv')

In [13]:
test = pd.read_csv('./output/clean_NSC dp TP_15.50.18 RF091120.csv')
test

FileNotFoundError: [Errno 2] File ./output/clean_NSC dp TP_15.50.18 RF091120.csv does not exist: './output/clean_NSC dp TP_15.50.18 RF091120.csv'

In [ ]:
nav.head(20)

In [ ]:
rel_data.head(20)

In [ ]:
i = 1
rel_data.iloc[0:2,num_param*(i-1) + (5-1)]

In [ ]:
print("{:03}".format(10))

In [ ]:
plate = %run -i "grouper.py"

In [ ]:
print(plate)

In [1]:
import os
import sys

sys.path.append(os.getcwd())
import grouper


What is your group layout?
How many rows per group? 4
How many columns per group? 24
(1) NSC dp TP_15.50.18 RF091120.csv
(2) NSC_IV_ppm120_18.13.14 RF091120.csv
(3) NSC_IV_ppm120_15.51.52 RF110920.csv
(4) Nav18_virtSSI_15.53.46 RF091120.csv
Which file do you want to process? 2
Getting relevant data.
Skipping Seal Resistance
Processing INa
Enter the name for group A1: First
Enter the name for group A2: Second
Enter the name for group A3: Third
Enter the name for group A4: Fourth
Processing IK
Processing Itail
Processing Capacitance
Processing Sweep VoltagePerCurrent
Skipping Sweep disregarded


In [9]:
plate = grouper.Plate()
grouper.process_file(plate())


What is your group layout?
How many rows per group? 4
How many columns per group? 24


TypeError: 'Plate' object is not callable

In [24]:
for_fit = pd.to_numeric(plate.get_potentials())
print(for_fit)
for_fit = [x*1000 for x in for_fit]
print(for_fit)
xdata = np.asarray(for_fit)

[]
[]


In [25]:
grouper.process_file(plate)

(1) NSC dp TP_15.50.18 RF091120.csv
(2) NSC_IV_ppm120_18.13.14 RF091120.csv
(3) NSC_IV_ppm120_15.51.52 RF110920.csv
(4) Nav18_virtSSI_15.53.46 RF091120.csv
Which file do you want to process? 
Please enter a number: 
Please enter a number: 
Please enter a number: 
Please enter a number: 
Please enter a number: 
Please enter a number: 
Please enter a number: 


KeyboardInterrupt: Interrupted by user

In [7]:
ydata = np.zeros(shape=plate.num_sweeps)
frame_name = "Itail" # Parameter names shifted to the right
for sweep in range(1, plate.num_sweeps + 1):
    curr = frame_name + "{:03d}".format(sweep)
    ydata[sweep - 1] = plate.get_param_frame(curr).astype('float64').iloc[1,1]
print(xdata.size, ydata.size)
ydata = np.asarray([y*1000000000000 for y in ydata])
ydata = np.divide(ydata, np.max(ydata))
for point in range(ydata.size):
    print("Potential: {} Current: {}".format(xdata[point],ydata[point]))

NameError: name 'plate' is not defined

In [20]:
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

In [21]:
def func_IV(v, gmax, vhalf, vslope): # IV
    return gmax/(1 + np.exp((vhalf - v)/vslope)) #(v - vrev)*

def func_IV_NA(v, gmax, vhalf, vslope): # IV
    return (v - vrev) * gmax/(1 + np.exp((vhalf - v)/vslope))

def func_SSI(v, fmc, vhalf, z1): # Probabaly:
    return fmc/(1 + np.exp((vhalf - v)/(25.3*z1)))

In [22]:
plt.rcParams['figure.figsize'] = [10, 5]
xdata = pd.to_numeric(plate.get_potentials())

In [23]:
popt, pcov = curve_fit(sigmoid, xdata, ydata) # Request initials
popt2, pcov2 = curve_fit(func_IV, xdata, ydata, p0=[20,10,10], maxfev=1000000000)
popt3, pcov3 = curve_fit(func_SSI, xdata, ydata, p0=[20,10,10], maxfev=1000000000)
plt.plot(xdata, ydata, 'b.', label="data")
xrange = np.arange(np.min(xdata), np.max(xdata), 0.01)
plt.plot(xrange, func_IV(xrange, *popt2), 'r-', label='fit: gmax=%5.3f, vhalf=%5.3f, vslope=%5.3f' % tuple(popt2))
plt.plot(xrange, func_SSI(xrange, *popt3), 'y-', label='fit: fmc=%5.3f, vhalf=%5.3f, z1=%5.3f' % tuple(popt3))

plt.legend()
plt.xlabel("Potential")
plt.ylabel("Current")
plt.title("IV")
v = popt[0]

NameError: name 'sigmoid' is not defined

In [ ]:
@interact(car=1)
def f(car):
    return car

In [144]:
def sigmoid(x,V):
    return (1/(1+np.exp(-x/V)))